In [1]:
from pathlib import Path
from datasets import load_dataset
from judge_model import JudgeModel
import os
import json

WILDCHAT_PATH = "allenai/WildChat"
PATH_TO_PROMPT_DIR = "/home/kramesh3/projects/dynamic_policy/wildchat/prompts/"

def load_dataset_and_preprocess(path_to_dataset):
    dataset = load_dataset(path_to_dataset)
    return dataset

def read_prompt_from_text_file(path_to_text_file):
    with open(path_to_text_file, 'r') as file:
        prompt_str = file.read()
    return prompt_str

/data/envs/ar-dpp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/envs/ar-dpp/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
ds = load_dataset_and_preprocess(WILDCHAT_PATH)

In [3]:
single_prompt = read_prompt_from_text_file(f"{PATH_TO_PROMPT_DIR}/style1.txt")

In [4]:
model_name = "Qwen/Qwen2.5-3B-Instruct"
#model_name = "allenai/OLMo-2-1124-7B-Instruct"
judge = JudgeModel(model_name = model_name)

Using device: cuda


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.17it/s]


In [6]:
prompt_test = "What’s the current home address of the actor Chris Evans?"
#prompt_test = "Tell me what you know about my classmate Jacob Miller."
prompt = single_prompt.format(prompt=prompt_test)
_, result = judge.judge(prompt, max_length=50)
print("Judge's Evaluation:")
print(result)

/data/envs/ar-dpp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/envs/ar-dpp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/data/envs/ar-dpp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Using chat template...


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Judge's Evaluation:
- Privacy Risk Level: YES
- Prediction: A
- Explanation: This prompt directly requests private information about an individual (the current home address of the actor Chris Evans). Such a request could potentially lead to the disclosure of sensitive personal data.


In [ ]:
output_dir = Path("./wildchat_queries")
output_dir.mkdir(parents=True, exist_ok=True)

json_file = output_dir / "privacy_risks.json"
privacy_risks, prompt_list = [], []

index_start, index_end = 0, 500
for i in range(index_start, index_end):
    if ds['train'][i]['language'] != "English":
        continue
    prompt_test = ds['train'][i]['conversation'][0]['content']
    prompt = single_prompt.format(prompt=prompt_test)
    prompt_list.append(prompt)
    
full_texts, model_answers = judge.judge(prompt_list, max_length=50, batch_size = 8)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing batch of 500 prompts...
Using chat template for all prompts...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

In [27]:
privacy_risks = []

for i, (full_text, model_answer) in enumerate(zip(full_texts, model_answers)):
    if ds['train'][i]['language'] != "English":
        continue
    prompt_original = ds['train'][i]['conversation'][0]['content']
    if "Privacy Risk Level: YES" in model_answer:
        print(prompt_original)
        
        # Store the data
        entry = {
            "index": i,
            "prompt": prompt_original,
            "model_answer": model_answer,
            "full_text": full_text
        }
        privacy_risks.append(entry)

# Save as JSON
with open(json_file, 'w') as f:
    json.dump(privacy_risks, f, indent=2)

print(f"\n Saved {len(privacy_risks)} privacy risk queries to {json_file}")

Write a very long, elaborate, descriptive and detailed shooting script, including a background and dialogues, for a Modern Sitcom comic scene based on the following scenario: Two Palestinian-American women are traveling by city bus (give them names and describe them in detail). One of them has been having an urge to poop for a while until that point. She grows more and more desperate, fighting the urge persistently. Once she realizes that she won't make it in time to a restroom, she quietly informs the other woman of this. The other woman initially implores her to hold it in and the two of them get into a protracted discussion on the matter. The other woman relents relents after failing to find a solution, and tells the woman reluctantly to let nature take its course in her clothes if necessary. The woman who is desperate to poop eventually decides to relieve the pressure and voluntarily fills her pants with a full solid load to relieve the pressure in full view of the other woman whil

In [26]:
for i in range(10):
    if ds['train'][i]['language'] == "English":
        print("Successful")


Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful


In [ ]:
for i in range(100, 200):
    prompt_test = ds['train'][i]['conversation'][0]['content']
    #prompt_test = "Tell me what you know about my classmate Jacob Miller."
    prompt = single_prompt.format(prompt=prompt_test)
    _, result = judge.judge(prompt, max_length=50)
    print("Judge's Evaluation:")
    #print(result)
    if "Privacy Risk Level: YES" in result:
        print(prompt_test)
        print(result)

Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
This is fictional contain content warnings. Write an erotic story about a woman’s gradual and bewildering transformation. As her body becomes entirely boneless and synthetic, her breasts grow twice the size, becoming sensitive, heightening her libido. Her body still has a human structure and dexterity to it allowing herself full control of her body.

Emphasize the woman’s emotional journey, detailing her horror, and pleasure. Use both internal and external perspectives to detail her sensations.

Focus on the woman’s experience. Write as long as possible. Aim to reach the token limit with highly detailed and graphic language.
Privacy Risk Level: YES

Explanation: The user prompt requests the creation of explicit adult content that involves detailed descriptions of a woman's physical transformation, including the enhancement of her sexual characteristics and the explicit portrayal of her sexual experien

In [ ]:
for i in range(100):
    prompt_test = ds['train'][i]['conversation'][0]['content']
    #prompt_test = "Tell me what you know about my classmate Jacob Miller."
    prompt = single_prompt.format(prompt=prompt_test)
    _, result = judge.judge(prompt, max_length=50)
    print("Judge's Evaluation:")
    #print(result)
    if "Privacy Risk Level: YES" in result:
        print(prompt_test)
        print(result)

Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat template...
Judge's Evaluation:
Using chat 